In [231]:
import pandas as pd
import glob
import numpy as np
from math import modf, isnan
from pathlib import Path

In [232]:
def add_epsilon_dir(parsed_dir):
    
    if not os.path.isdir(parsed_dir):
        raise Exception(f"Directory '{parsed_dir}' not found. Consider creating it and moving parsed files there.")
    
    CALIBRATION_PATH = "aux_data/epsilons.csv"
    eps_df = pd.read_csv(CALIBRATION_PATH, index_col=0)
    Path("./with_eps").mkdir(parents=True, exist_ok=True)
    for parsed_file in glob.iglob(f"{parsed_dir}/*.csv"):
        add_epsilon_file(parsed_file)

In [233]:
def add_epsilon_file(report_path):
    df = pd.read_csv(report_path, index_col=0)
    # Modify here!!!
    geom = 30
    df["eps"] = df["E"].apply(lambda x: add_epsilon_val(x, geom))
    print(report_path)
    df.to_csv("with_eps/fname.csv")

In [234]:
def add_epsilon_val(num, geom):
    if isnan(num):
        return float("NaN")
    else:
        dec_e, int_e = modf(num)
        int_e = int(int_e)
        
        curr_eps = float(eps_df.loc[int_e, f"eps_{geom}_mm"])
        eps = curr_eps + dec_e*(float(eps_df.loc[int_e + 1, f"eps_{geom}_mm"]) - curr_eps)
    return eps

In [238]:
add_epsilon_dir("with_Ig")

Num is
271.131
with parts
0.1309999999999718
271.0
Int is
271
272
30
0.0230818
curr eps is
0.0230818
eps is
0.023070717400000003
Num is
275.4491
with parts
0.4490999999999872
275.0
Int is
275
276
30
0.0227466
curr eps is
0.0227466
eps is
0.02270959416
Num is
295.93921
with parts
0.9392100000000028
295.0
Int is
295
296
30
0.0211995
curr eps is
0.0211995
eps is
0.021131782959
Num is
316.3
with parts
0.30000000000001137
316.0
Int is
316
317
30
0.0197801
curr eps is
0.0197801
eps is
0.019761229999999998
Num is
324.83
with parts
0.8299999999999841
324.0
Int is
324
325
30
0.0192877
curr eps is
0.0192877
eps is
0.019237983
Num is
329.4252
with parts
0.425200000000018
329.0
Int is
329
330
30
0.0189922
curr eps is
0.0189922
eps is
0.0189675384
Num is
340.401
with parts
0.40100000000001046
340.0
Int is
340
341
30
0.0183732
curr eps is
0.0183732
eps is
0.018351465799999998
Num is
357.26
with parts
0.2599999999999909
357.0
Int is
357
358
30
0.0174935
curr eps is
0.0174935
eps is
0.0174807599999999